### [FlashRank](https://github.com/PrithivirajDamodaran/FlashRank) 是一个超轻量、超快速的库，用于向现有的搜索和检索流程中添加重新排名功能。它基于最先进的交叉编码器技术，这使得它能够在处理大量数据时保持高效率。

%pip install --upgrade --quiet flashrank  
%pip install --upgrade --quiet faiss  
如果只需要cpu版本  
%pip install --upgrade --quiet faiss_cpu

In [1]:
##打印文本内容
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

### 我们开始初始化一个简单的向量存储检索器

以[baichuan2](https://arxiv.org/pdf/2309.10305v2.pdf)论文为例 ， 我们可以设置检索器以检索大量（20个）文档。

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = PyPDFLoader("https://arxiv.org/pdf/2309.10305.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents[:15])
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
retriever = FAISS.from_documents(texts, embedding).as_retriever(search_kwargs={"k": 20})
query = "What is Baichuan2？"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

compared to Baichuan 1-7B. Specifically, Baichuan
2 is optimized to improve performance on math
and code problems. On the GSM8K (Cobbe
et al., 2021) and HumanEval (Chen et al., 2021)
evaluations, Baichuan 2 nearly doubles the results
of the Baichuan 1. In addition, Baichuan 2 also
demonstrates strong performance on medical and
legal domain tasks. On benchmarks such as
MedQA (Jin et al., 2021) and JEC-QA (Zhong
et al., 2020), Baichuan 2 outperforms other open-
Metadata: {'source': 'https://arxiv.org/pdf/2309.10305.pdf', 'page': 1, 'id': 14}
----------------------------------------------------------------------------------------------------
Document 2:

to Baichuan 1-7B, Baichuan 2-7B-Base shows an
improvement of nearly 10 points. In the medical
field, Baichuan 2-7B-Base outperforms models
like ChatGLM 2-6B and LLaMA 2-7B, showing
significant improvement over Baichuan 1-7B as
well.
Similarly, Baichuan 2-13B-Base surpasses
models other than GPT-4 in the field of Chinese
law. 

### 使用FlashRank进行重排

现在让我们用一个**ContextualCompressionRetriever**来包装我们的基础检索器，并使用**FlashrankRerank**作为压缩器。

In [4]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What is Baichuan2？"
)
print([doc.metadata["id"] for doc in compressed_docs])

Running pairwise ranking..
[9, 1, 19]


重排序之后，top3 已经和之前的top3发生了很大的变化

In [5]:
pretty_print_docs(compressed_docs)

Document 1:

350 iterations for all our chat models, resulting in
Baichuan 2-7B-Chat and Baichuan 2-13B-Chat.
Metadata: {'id': 9, 'relevance_score': [0.9729050397872925, 0.06210247427225113]}
----------------------------------------------------------------------------------------------------
Document 2:

to Baichuan 1-7B, Baichuan 2-7B-Base shows an
improvement of nearly 10 points. In the medical
field, Baichuan 2-7B-Base outperforms models
like ChatGLM 2-6B and LLaMA 2-7B, showing
significant improvement over Baichuan 1-7B as
well.
Similarly, Baichuan 2-13B-Base surpasses
models other than GPT-4 in the field of Chinese
law. In the medical domain, Baichuan 2-13B-
Base outperforms models such as XVERSE-13B
Metadata: {'id': 1, 'relevance_score': [0.9596770405769348, 0.08595798164606094]}
----------------------------------------------------------------------------------------------------
Document 3:

a variety of natural language tasks based
on just a few examples of natural language
inst

In [6]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [7]:
chain.invoke(query)

Running pairwise ranking..


{'query': 'What is Baichuan2？',
 'result': 'Baichuan 2 is a series of large-scale multilingual language models containing 7 billion and 13 billion parameters. These models were trained from scratch on 2.6 trillion tokens and are designed to perform well on a variety of natural language tasks without the need for extensive feature engineering. Baichuan 2 has shown significant improvements in various fields such as medicine and Chinese law compared to other models like ChatGLM and LLaMA.'}